# ray_deploy.ipynb

Optimized model serving implementation from the [benchmark notebook](./benchmark.ipynb).

## Boilerplate

In [1]:
# Initialization and import code goes in this cell.

# Imports: Python core, then third-party, then local.
# Try to keep each block in alphabetical order, or the linter may get angry.

import asyncio
import requests
import starlette
import time
import os
import json

import scipy.special

import ray
from ray import serve
import torch
import transformers
import zerocopy

import concurrent

# Fix silly warning messages about parallel tokenizers
os.environ['TOKENIZERS_PARALLELISM'] = 'False'


# Reduce the volume of warning messages from `transformers`
transformers.logging.set_verbosity_error()


def reboot_ray():
    if ray.is_initialized():
        ray.shutdown()

    if torch.cuda.is_available():
        return ray.init(num_gpus=1)
    else:
        return ray.init()

In [2]:
# Constants go here
INTENT_MODEL_NAME = 'mrm8488/t5-base-finetuned-e2m-intent'
SENTIMENT_MODEL_NAME = 'cardiffnlp/twitter-roberta-base-sentiment'
QA_MODEL_NAME = 'deepset/roberta-base-squad2'
GENERATE_MODEL_NAME = 'gpt2'


INTENT_INPUT = {
    'context':
        ("I came here to eat chips and beat you up, "
         "and I'm all out of chips.")
}

SENTIMENT_INPUT = {
    'context': "We're not happy unless you're not happy."
}

QA_INPUT = {
    'question': 'What is 1 + 1?',
    'context': 
        """Addition (usually signified by the plus symbol +) is one of the four basic operations of 
        arithmetic, the other three being subtraction, multiplication and division. The addition of two 
        whole numbers results in the total amount or sum of those values combined. The example in the
        adjacent image shows a combination of three apples and two apples, making a total of five apples. 
        This observation is equivalent to the mathematical expression "3 + 2 = 5" (that is, "3 plus 2 
        is equal to 5").
        """
}

GENERATE_INPUT = {
    'prompt_text': 'All your base are'
}

## Example model code

This is the single-node code on which the Serve deployments below are based.  Some of this code is duplicated in `benchmark.ipynb` and should be kept in sync.

### Intent model

For our intent detection models, we'll use the model [`mrm8488/t5-base-finetuned-e2m-intent`](https://huggingface.co/mrm8488/t5-base-finetuned-e2m-intent).

The intent model comes as three parts: A *tokenizer* that converts raw text into a sequence numeric token IDs, a core *model* that transforms these token sequences, and *preprocessing and postprocessing code* to choreograph the usage of the first two parts.

In [3]:
# This code block also appears in `benchmark.ipynb`

# Load model and tokenizer
intent_tokenizer = transformers.AutoTokenizer.from_pretrained('t5-base')
intent_model = transformers.AutoModelForSeq2SeqLM.from_pretrained(
    INTENT_MODEL_NAME)

# Preprocessing
input_text = f'{INTENT_INPUT["context"]} </s>'
features = intent_tokenizer([input_text], return_tensors='pt')

# Inference
output = intent_model.generate(**features)

# Postprocessing
result_string = intent_tokenizer.decode(output[0])
result_string = result_string.replace('<pad>', '')
result_string = result_string[len(' '):-len('</s>')]

result_string

'to eat'

### Sentiment model

For our sentiment models, we'll use model [`cardiffnlp/twitter-roberta-base-sentiment`](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment).

Like the intent model, the sentiment model is packaged as a tokenizer, a core model, and instructions for pre- and post-processing.

In [4]:
# Model loading
sentiment_tokenizer = transformers.AutoTokenizer.from_pretrained(
    SENTIMENT_MODEL_NAME)
sentiment_model = (transformers.AutoModelForSequenceClassification
                .from_pretrained(SENTIMENT_MODEL_NAME))

# Preprocessing
encoded_input = sentiment_tokenizer(SENTIMENT_INPUT['context'], 
                                 return_tensors='pt')   

# Inference
output = sentiment_model(**encoded_input)

# Postprocessing
scores = output[0][0].detach().numpy()
scores = scipy.special.softmax(scores)
scores = [float(s) for s in scores]
scores = {k: v for k, v in zip(['positive', 'neutral', 'negative'], scores)}

scores

{'positive': 0.5419477820396423,
 'neutral': 0.38251084089279175,
 'negative': 0.07554134726524353}

### Question Answering Model

For our question answering models, we'll use the model [`deepset/roberta-base-squad2`](https://huggingface.co/deepset/roberta-base-squad2).

Unlike the intent and sentiment models, the question answering model comes prepackaged as a `question-answering` pipeline via the `tokenizers` library's [Pipelines API](https://huggingface.co/docs/transformers/main_classes/pipelines). 

So we can load and run all parts of the model, including pre- and post-processing code, by creating an instance of the pipeline class. The pipeline object has methods `preprocess()`, `forward()`, and `postprocess()` to perform preprocessing, inference, and postprocessing.

In [5]:
# Loading the model
qa_pipeline = transformers.pipeline('question-answering',
                                    model=QA_MODEL_NAME)
# Preprocessing (returns a Python generator)
qa_pre = qa_pipeline.preprocess(qa_pipeline.create_sample(**QA_INPUT))

# Inference
qa_output = (qa_pipeline.forward(example) for example in qa_pre)

# Postprocessing
qa_result = qa_pipeline.postprocess(qa_output)

qa_result

{'score': 4.278938831703272e-06, 'start': 483, 'end': 484, 'answer': '5'}

There is also a convenience method `__call__()` that runs all three phases of processing in sequence..

In [6]:
# This code also appears in `benchmark.ipynb`

# Loading the model and associated resources
qa_pipeline = transformers.pipeline('question-answering',
                                    model=QA_MODEL_NAME)
# Preprocessing, inference, and postprocessing all happen in
# the Python object's the __call__() method.
qa_result = qa_pipeline(**QA_INPUT)

qa_result

{'score': 4.278938831703272e-06, 'start': 483, 'end': 484, 'answer': '5'}

### Natural Language Generation Model

For natural language generation, we'll use the [`gpt2`](https://huggingface.co/gpt2) language model. Like the question answering model, this natural language generation model comes wrapped in a `tokenizers` pipeline class. The class's `__call__()` method performs all the steps necessary to run end-to-end inference.


In [7]:
# Load the model
generate_pipeline = transformers.pipeline(
    'text-generation', model=GENERATE_MODEL_NAME)
pad_token_id = generate_pipeline.tokenizer.eos_token_id

# Preprocessing
generate_pre = generate_pipeline.preprocess(**GENERATE_INPUT)

# Inference
generate_output = generate_pipeline.forward(generate_pre,
                                            pad_token_id=pad_token_id)

# Postprocessing
generate_result = generate_pipeline.postprocess(generate_output)
generate_result

[{'generated_text': 'All your base are all mine\n\n"This means there are not many of mine."\n\nOne of my sisters has a knife and she\'d been thinking of me. She\'d got this over with, just because the knife was not on my'}]

## Start Ray Serve

In [8]:
serve.shutdown()
reboot_ray()
serve.start()

2022-03-02 11:35:18,780	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265
(ServeController pid=6451) 2022-03-02 11:35:23,196	INFO checkpoint_path.py:16 -- Using RayInternalKVStore for controller checkpoint and recovery.
(ServeController pid=6451) 2022-03-02 11:35:23,308	INFO http_state.py:98 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SioqGl:SERVE_PROXY_ACTOR-node:127.0.0.1-0' on node 'node:127.0.0.1-0' listening on '127.0.0.1:8000'
2022-03-02 11:35:23,769	INFO api.py:475 -- Started Serve instance in namespace '8bf422d7-bce3-4383-b4fe-28ad2a215978'.


## Optimized Model Deployments

Some of these classes appear in slightly modified format in `benchmark.ipynb`. Make sure to keep the code in sync.

In [9]:
# This class also appears in `benchmark.ipynb`
@serve.deployment
class Intent:
    def __init__(self):
        self._tokenizer = transformers.AutoTokenizer.from_pretrained('t5-base')
        model = transformers.AutoModelForSeq2SeqLM.from_pretrained(
                    INTENT_MODEL_NAME)
        
        # Extract weights and load them onto the Plasma object store
        self._model_ref = ray.put(zerocopy.extract_tensors(model))

    async def __call__(self, request: starlette.requests.Request):
        json_request = await request.json()
        
        # Preprocessing
        input_text = f'{json_request["context"]} </s>'
        features = self._tokenizer([input_text], return_tensors='pt')

        # Model inference runs asynchronously in a Ray task
        output = await zerocopy.call_model.remote(
            self._model_ref, [], features, 'generate')

        # Postprocessing
        result_string = self._tokenizer.decode(output[0])
        result_string = result_string[len('<pad> '):-len('</s>')]
        return {
            'intent': result_string
        }


@serve.deployment
class Sentiment:
    def __init__(self):
        self._tokenizer = transformers.AutoTokenizer.from_pretrained(
            SENTIMENT_MODEL_NAME)
        
        model = (transformers.AutoModelForSequenceClassification
                 .from_pretrained(SENTIMENT_MODEL_NAME))
        self._model_ref = ray.put(zerocopy.extract_tensors(model))

    async def __call__(self, request: starlette.requests.Request):
        json_request = await request.json()
        
        # Preprocessing
        encoded_input = self._tokenizer(json_request['context'], 
                                         return_tensors='pt')   

        # Inference
        output = await zerocopy.call_model.remote(
            self._model_ref, [], encoded_input)

        # Postprocessing
        scores = output[0][0].detach().numpy()
        scores = scipy.special.softmax(scores)
        scores = [float(s) for s in scores]
        scores = {k: v for k, v in zip(['positive', 'neutral', 'negative'], scores)}
        return scores


# This class also appears in `benchmark.ipynb`
@serve.deployment
class QA:
    def __init__(self):
        # Load the pipeline and move the model's weights onto the
        # Plasma object store.
        self._pipeline = zerocopy.rewrite_pipeline(
            transformers.pipeline('question-answering', 
                                  model=QA_MODEL_NAME))
        self._threadpool = concurrent.futures.ThreadPoolExecutor()

    async def __call__(self, request: starlette.requests.Request):
        json_request = await request.json()

        # The original `transformers` code is not async-aware, so we
        # call it from `run_in_executor()`
        result = await asyncio.get_running_loop().run_in_executor(
             self._threadpool, lambda: self._pipeline(**json_request))
        return result
    

@serve.deployment
class Generate:
    def __init__(self):
        self._pipeline = zerocopy.rewrite_pipeline(
            transformers.pipeline('text-generation',
                                  model=GENERATE_MODEL_NAME),
            ('__call__', 'generate'))
        self._pad_token_id = self._pipeline.tokenizer.eos_token_id
        self._threadpool = concurrent.futures.ThreadPoolExecutor()

    async def __call__(self, request: starlette.requests.Request):
        json_request = await request.json()

        result = await asyncio.get_running_loop().run_in_executor(
            self._threadpool, 
            lambda: self._pipeline(
                json_request['prompt_text'], 
                pad_token_id=self._pad_token_id))
        return result


(HTTPProxyActor pid=6452) INFO:     Started server process [6452]


Now we can deploy all of these pipelines as Serve endpoints.

In [10]:
# Define endpoints.
# Everything gets deployed under the prefix /predictions/ to make
# the deployment as similar as possible to the TorchServe baseline.
LANGUAGES = ['en', 'es', 'zh']

# We run the deployments in a blocking fashion for safety.
for lang in LANGUAGES:
    Intent.options(name=f'intent_{lang}',
                   route_prefix=f'/predictions/intent_{lang}',
                   ray_actor_options={"num_cpus": 0.1}).deploy()
    Sentiment.options(name=f'sentiment_{lang}',
                   route_prefix=f'/predictions/sentiment_{lang}',
                   ray_actor_options={"num_cpus": 0.1}).deploy()
    QA.options(name=f'qa_{lang}',
                   route_prefix=f'/predictions/qa_{lang}',
                   ray_actor_options={"num_cpus": 0.1}).deploy()
    Generate.options(name=f'generate_{lang}',
                   route_prefix=f'/predictions/generate_{lang}',
                   ray_actor_options={"num_cpus": 0.1}).deploy()

# Wait a moment so log output doesn't go to the next cell's output
time.sleep(1.)

2022-03-02 11:35:23,817	INFO api.py:249 -- Updating deployment 'intent_en'. component=serve deployment=intent_en
(ServeController pid=6451) 2022-03-02 11:35:23,845	INFO deployment_state.py:920 -- Adding 1 replicas to deployment 'intent_en'. component=serve deployment=intent_en
(intent_en pid=6450) The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
2022-03-02 11:35:34,414	INFO api.py:261 -- Deployment 'intent_en' is ready at `http://127.0.0.1:8000/predictions/intent_en`. component=serve deployment=intent_en
2022-03-02 11:35:34,426	INFO api.py:249 -- Updating deployment 'sentiment_en'. component=serve deployment=sentiment_en
(ServeController pid=6451) 2022-03-02 11:35:34,519	INFO deployment_state.py:920 -- Adding 1 replicas to deployment 'sentiment_en'. component=serve deployment=sentiment_en
2022-03-02 11:35:43,494	INFO api.py:261 -- Deployment 'sentiment_en' is ready at `http://127.0.0.1:8000

In [12]:
# Verify that everything deployed properly.
intent_result = requests.put(
    'http://127.0.0.1:8000/predictions/intent_en', 
    json.dumps(INTENT_INPUT)).json()
print(f'Intent result: {intent_result}')

sentiment_result = requests.put(
    'http://127.0.0.1:8000/predictions/sentiment_en', 
    json.dumps(SENTIMENT_INPUT)).json()
print(f'Sentiment result: {sentiment_result}')

qa_result = requests.put(
    'http://127.0.0.1:8000/predictions/qa_en', 
    json.dumps(QA_INPUT)).json()
print(f'Question answering result: {qa_result}')

generate_result = requests.put(
    'http://127.0.0.1:8000/predictions/generate_en', 
    json.dumps(GENERATE_INPUT)).json()
print(f'Natural language generation result: {generate_result}')

Intent result: {'intent': 'to eat'}
Sentiment result: {'positive': 0.5419477820396423, 'neutral': 0.38251084089279175, 'negative': 0.07554134726524353}
Question answering result: {'score': 4.278938831703272e-06, 'start': 483, 'end': 484, 'answer': '5'}
Natural language generation result: [{'generated_text': "All your base are going to change. You'll have to adjust your order accordingly.\n\nOnce your order is received and approved, you will then be able to order your next order. We want you to be able to read all about our services"}]


# Cleanup

Once the benchmark is complete, shut down this notebook's Ray cluster.

In [13]:
serve.shutdown()
ray.shutdown()

(ServeController pid=6451) 2022-03-02 11:51:16,917	INFO deployment_state.py:940 -- Removing 1 replicas from deployment 'intent_en'. component=serve deployment=intent_en
(ServeController pid=6451) 2022-03-02 11:51:16,922	INFO deployment_state.py:940 -- Removing 1 replicas from deployment 'sentiment_en'. component=serve deployment=sentiment_en
(ServeController pid=6451) 2022-03-02 11:51:16,925	INFO deployment_state.py:940 -- Removing 1 replicas from deployment 'qa_en'. component=serve deployment=qa_en
(ServeController pid=6451) 2022-03-02 11:51:16,929	INFO deployment_state.py:940 -- Removing 1 replicas from deployment 'generate_en'. component=serve deployment=generate_en
(ServeController pid=6451) 2022-03-02 11:51:16,932	INFO deployment_state.py:940 -- Removing 1 replicas from deployment 'intent_es'. component=serve deployment=intent_es
(ServeController pid=6451) 2022-03-02 11:51:16,936	INFO deployment_state.py:940 -- Removing 1 replicas from deployment 'sentiment_es'. component=serve de